# Preparazione all'asta di Fantacalcio 2024-2025

Obiettivi:
* quotazioni di giocatori diversi.
* ...

## Getting started

In [201]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

files = [
    "../data/Quotazioni_Fantacalcio_Stagione_2024_25.xlsx",
    "../data/Statistiche_Fantacalcio_Stagione_2023_24.xlsx",
    "../data/Quotazioni_FantacalcioOnline.xlsx"
]

quote = pd.read_excel(files[0], header = 1)
stats = pd.read_excel(files[1], header = 1)

In [202]:
def remove_weird_end(s):
    out = s.str.replace(" [A-Z].$", "").str.replace(" [A-Z][a-z].$", "")
    return out

quote["Nome"] = remove_weird_end(quote["Nome"])
stats["Nome"] = remove_weird_end(stats["Nome"])

merged = pd.merge(quote, stats, on = ["Nome", "Squadra", "R", "Id"], how = "right")

if "Qt.A" in merged.columns:
    merged = merged.drop(["Qt.A", "Qt.I", "Diff.", "Qt.I M", "FVM M", "Gf", "Gs", "Rp", "Rc", "R+", "R-"], axis = 1)
merged.head(6)

<ipython-input-202-c5f54ab8df05>:2: FutureWarning:

The default value of regex will change from True to False in a future version.



,Id,R,RM,Nome,Squadra,Qt.A M,Diff.M,FVM,Rm,Pv,Mv,Fm,Ass,Amm,Esp,Au
0,2428,P,Por,Sommer,Inter,19.0,1.0,110.0,Por,34,6.12,5.63,0,1,0,0
1,453,P,NaN,Szczesny,Juventus,NaN,NaN,NaN,Por,35,6.21,5.43,0,1,0,0
2,5841,P,Por,Svilar,Roma,14.0,1.0,70.0,Por,15,6.47,5.37,0,1,0,0
3,5876,P,NaN,Di Gregorio,Monza,NaN,NaN,NaN,Por,33,6.35,5.47,0,0,0,0
4,133,P,Por,Skorupski,Bologna,12.0,1.0,50.0,Por,32,6.17,5.33,0,2,0,0
5,2815,P,Por,Terracciano,Fiorentina,11.0,0.0,20.0,Por,33,6.27,5.26,0,3,0,0


In [199]:
# Load eval
eval  = pd.read_excel(files[2], header = 1)

# Keep only uppercase letters in "Nome"
eval["Nome"] = eval["Nome"].str.replace("[a-z]", "").str.replace(" [A-Z]$", "").str.title()
eval = eval[(eval.Posizione != "POR") | (eval.Nome != "Martinez")]
eval["Nome"] = eval["Nome"].str.replace(" ..$", "")
eval["Nome"] = eval["Nome"].str.replace(" [A-Z]$", "")
eval["Nome"] = eval["Nome"].str.replace(" -$", "")

<ipython-input-199-381de0cce35b>:5: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-199-381de0cce35b>:7: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-199-381de0cce35b>:8: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-199-381de0cce35b>:9: FutureWarning:

The default value of regex will change from True to False in a future version.



In [215]:
eval2 = eval[["Nome", "Squadra", "RAT", "IS %", "Kapitals"]].rename(columns={"RAT":"FantaRating", "IS %": "Schierabilita"})
merged2 = pd.merge(eval2, merged, on = ["Nome", "Squadra"], how="outer")
del eval2
merged2.head(4)

# jitter shierabilita
merged2["Schierabilita"] = merged2["Schierabilita"] + np.random.normal(0, 2, merged2.shape[0])

In [216]:
missing1 = merged2[np.isnan(merged2.Id)].sort_values("Nome")
print(missing1.shape)
print(missing1.head())

missing2 = merged2[np.isnan(merged2.FantaRating)].sort_values("Nome")
print(missing2.shape)
print(missing2.head())

(271, 19)
           Nome     Squadra  FantaRating  Schierabilita  Kapitals  Id    R  \
304  Abdulhamid        Roma         58.0      55.239905      10.0 NaN  NaN   
68      Abraham       Milan         62.0      62.235653      19.0 NaN  NaN   
545   Accornero       Genoa         45.0      11.952325       1.0 NaN  NaN   
49        Adams      Torino         66.0      74.556798      22.0 NaN  NaN   
286        Adli  Fiorentina         59.0      64.319465      10.0 NaN  NaN   

      RM  Qt.A M  Diff.M  FVM   Rm  Pv  Mv  Fm  Ass  Amm  Esp  Au  
304  NaN     NaN     NaN  NaN  NaN NaN NaN NaN  NaN  NaN  NaN NaN  
68   NaN     NaN     NaN  NaN  NaN NaN NaN NaN  NaN  NaN  NaN NaN  
545  NaN     NaN     NaN  NaN  NaN NaN NaN NaN  NaN  NaN  NaN NaN  
49   NaN     NaN     NaN  NaN  NaN NaN NaN NaN  NaN  NaN  NaN NaN  
286  NaN     NaN     NaN  NaN  NaN NaN NaN NaN  NaN  NaN  NaN NaN  
(386, 19)
         Nome   Squadra  FantaRating  Schierabilita  Kapitals      Id  R  \
704  Abankwah   Udinese    

In [217]:
def search_by_player(player):
    return pd.concat([
        missing2[missing2["Nome"] == player],
        missing1[missing1["Nome"] == player]
    ], ignore_index=True)

def search_by_team(team):
    return pd.concat([
        missing2[missing2["Squadra"] == team],
        missing1[missing1["Squadra"] == team]
    ], ignore_index=True)


search_by_player("Martinez")
search_by_team("Inter")


,Nome,Squadra,FantaRating,Schierabilita,Kapitals,Id,R,RM,Qt.A M,Diff.M,FVM,Rm,Pv,Mv,Fm,Ass,Amm,Esp,Au
0,Audero,Inter,NaN,NaN,NaN,761.0,P,NaN,NaN,NaN,NaN,Por,4.0,6.00,5.25,0.0,0.0,0.0,0.0
1,Correa,Inter,NaN,NaN,NaN,495.0,A,A,2.0,-1.0,7.0,A,0.0,0.00,0.00,0.0,0.0,0.0,0.0
2,Cuadrado,Inter,NaN,NaN,NaN,697.0,D,NaN,NaN,NaN,NaN,E,10.0,6.05,6.20,2.0,1.0,0.0,0.0
3,Esposito,Inter,NaN,NaN,NaN,4463.0,A,NaN,NaN,NaN,NaN,Pc,0.0,0.00,0.00,0.0,0.0,0.0,0.0
4,Gosens,Inter,NaN,NaN,NaN,2160.0,D,NaN,NaN,NaN,NaN,E,0.0,0.00,0.00,0.0,0.0,0.0,0.0
5,Klaassen,Inter,NaN,NaN,NaN,4182.0,C,NaN,NaN,NaN,NaN,C;T,6.0,6.00,6.00,0.0,1.0,0.0,0.0
6,Sanchez,Inter,NaN,NaN,NaN,2475.0,A,NaN,NaN,NaN,NaN,A,19.0,6.18,6.76,5.0,0.0,0.0,0.0
7,Sensi,Inter,NaN,NaN,NaN,1978.0,C,NaN,NaN,NaN,NaN,C;T,1.0,6.50,6.50,0.0,0.0,0.0,0.0
8,Stankovic,Inter,NaN,NaN,NaN,6248.0,P,NaN,NaN,NaN,NaN,Por,0.0,0.00,0.00,0.0,0.0,0.0,0.0
9,Palacios,Inter,58.0,55.345320,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### What do we need to plot?

Need to understand what Kapitals mean...

* is it always the case that k_350_* > k_500_*
* 

In [218]:
team_colors = {
    "Atalanta": "#1F4E79",  # Dark Blue
    "Bologna": "#A52A2A",  # Brown
    "Cagliari": "#C71585",  # Medium Violet Red
    "Empoli": "#006BB6",  # Royal Blue
    "Fiorentina": "#6A0DAD",  # Purple
    "Genoa": "#D91E18",  # Red
    "Inter": "#0033CC",  # Blue
    "Juventus": "#000000",  # Black
    "Lazio": "#87CEEB",  # Sky Blue
    "Lecce": "#FFD700",  # Gold
    "Monza": "#FF4500",  # Orange Red
    "Napoli": "#00FFFF",  # Cyan
    "Roma": "#800000",  # Maroon
    "Salernitana": "#8B0000",  # Dark Red
    "Sampdoria": "#4169E1",  # Royal Blue
    "Sassuolo": "#228B22",  # Forest Green
    "Torino": "#8B4513",  # Saddle Brown
    "Udinese": "#808080",  # Gray
    "Verona": "#FFFF00",  # Yellow
    "Milan": "#FF0000"  # Red
}

**Exploratory plots** 

Let us plot price over Mv:

In [219]:
# plot Qt.A over average vote (Fm)
fig = px.scatter(merged[merged["Fm"] > 0], 
                 x="Qt.A M", y="Fm",
                 hover_name="Nome",
                 color="Squadra",
                 facet_col= "R", facet_col_wrap=2,
                 color_discrete_map=team_colors,
                 title="Qt.A vs Fm")
fig

In [208]:
eval.columns
# px.scatter(eval, x = "")

Index(['Nome', 'RAT', 'POT', 'IS %', 'ETA'', 'Ruolo trequartista',
       'Ruolo standard', 'Posizione', 'Squadra', 'Kapitals', 'Bonus'],
      dtype='object')

In [209]:
print(merged2.columns)
print(eval.columns)

Index(['Nome', 'Squadra', 'FantaRating', 'Schierabilita', 'Kapitals', 'Id',
       'R', 'RM', 'Qt.A M', 'Diff.M', 'FVM', 'Rm', 'Pv', 'Mv', 'Fm', 'Ass',
       'Amm', 'Esp', 'Au'],
      dtype='object')
Index(['Nome', 'RAT', 'POT', 'IS %', 'ETA'', 'Ruolo trequartista',
       'Ruolo standard', 'Posizione', 'Squadra', 'Kapitals', 'Bonus'],
      dtype='object')


In [220]:
# plot Qt.A over average vote (Fm)
fig = px.scatter(merged2[ (merged2.R.notnull()) & (merged2.Fm > 5) ],
                 x="Schierabilita", y="Fm",
                 hover_name="Nome",
                 color="Squadra",
                 facet_col= "R", facet_col_wrap=2,
                 color_discrete_map=team_colors)
fig